In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nltk #install using "conda install anaconda::nltk"
import sklearn 
import string
import re # helps you filter urls

In [ ]:
# download NLTK models
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


lemmatizer= nltk.stem.wordnet.WordNetLemmatizer()
stopwords= stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/27abd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/27abd/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/27abd/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/27abd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/27abd/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
train = pd.read_csv("train.csv") 
train_tweets = train["Content"]
print(train_tweets.head(),'\n',len(train_tweets))

0    . and I pledge to be trusted partners who will...
1    Great to be with Liza Colón-Zayas to cook up a...
2    Kamala Harris wants to flood the United States...
3    Kamala Harris wants to raise taxes on American...
4    When President Trump is elected he will usher ...
Name: Content, dtype: object 
 1125


In [ ]:
# Test on one tweet
print(train_tweets[199])

#basic tokenization
tokens = word_tokenize(train_tweets[199])
print(tokens,'\n')

# remove emoji's
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
print(emoji_pattern.sub(r'', train_tweets[199])) # no emoji
tokens = word_tokenize(emoji_pattern.sub(r'', train_tweets[199])) 
print(tokens,'\n') #no emojis

Times Square was bejeweled this morning with our new ads 💎
['Times', 'Square', 'was', 'bejeweled', 'this', 'morning', 'with', 'our', 'new', 'ads', '💎'] 

Times Square was bejeweled this morning with our new ads 
['Times', 'Square', 'was', 'bejeweled', 'this', 'morning', 'with', 'our', 'new', 'ads'] 



In [ ]:
#tweet processing example
example_tweet = "🇺🇸 making world's best is what we're proud of doing off-the-shelf! And more proud:https://www.google.com"
#tempstr = gop_tweets[temp_num]
print(example_tweet)

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#handle url
print(re.sub(r'http\S+', '', example_tweet))
example_tweet = re.sub(r'http\S+', '', example_tweet)

#handle emojis
print(emoji_pattern.sub(r'', example_tweet)) # no emoji
example_tweet = emoji_pattern.sub(r'', example_tweet)

#handle apostrophe 's
print(example_tweet)
print(example_tweet.replace("'s","")) #replace 's to 
example_tweet = example_tweet.replace("'s","")

#handle other apostrophe
example_tweet = example_tweet.replace("'","")

#handle all other punctuation
print(re.sub(r'[' + string.punctuation + r']+', ' ', example_tweet).strip())
example_tweet = re.sub(r'[' + string.punctuation + r']+', ' ', example_tweet).strip()

#handle lowercase
example_tweet = example_tweet.lower()

tokens = word_tokenize(example_tweet)

print(tokens,'\n')

#lemmatize tokens
#lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()


#lemmatize single word or token
token_lemma = lemmatizer.lemmatize(tokens[0])
print(token_lemma)

#lemmatize a list of words or tokens
tweet_lemma = [lemmatizer.lemmatize(token) for token in tokens]
print(tweet_lemma)


🇺🇸 making world's best is what we're proud of doing off-the-shelf! And more proud:https://www.google.com
🇺🇸 making world's best is what we're proud of doing off-the-shelf! And more proud:
 making world's best is what we're proud of doing off-the-shelf! And more proud:
 making world's best is what we're proud of doing off-the-shelf! And more proud:
 making world best is what we're proud of doing off-the-shelf! And more proud:
making world best is what were proud of doing off the shelf  And more proud
['making', 'world', 'best', 'is', 'what', 'were', 'proud', 'of', 'doing', 'off', 'the', 'shelf', 'and', 'more', 'proud'] 

making
['making', 'world', 'best', 'is', 'what', 'were', 'proud', 'of', 'doing', 'off', 'the', 'shelf', 'and', 'more', 'proud']


In [ ]:
# Convert part of speech tag from nltk.pos_tag to word net compatible format

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN
    
def process(text, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ Normalizes case and handles punctuation
    Inputs:
        text: str: raw text
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs:
        list(str): tokenized text
    """
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

    #handle url
    processed = re.sub(r'http\S+', '', text)

    #handle emojis
    processed = emoji_pattern.sub(r'', processed)

    #handle apostrophe 'so 
    processed = processed.replace("'s","")

    #handle other apostrophe
    processed = processed.replace("'","")

    #handle all other punctuation
    processed = re.sub(r'[' + string.punctuation + r']+', ' ', processed).strip()

    #handle lowercase
    processed = processed.lower()

    # tokenize text
    tokens = word_tokenize(processed)

    # makes dictionary mapping token to treebank tag
    treebank_tags = nltk.pos_tag(tokens)

    # hold lemma tokens
    lemma_tokens = []

    for word, tree_tag in treebank_tags:
        # get corresponding wordnet tag from treebank tag
        wordnet_tag = get_wordnet_pos(tree_tag)

        # get lemma version of word and append to list
        lemma_token = lemmatizer.lemmatize(word, pos=wordnet_tag)
        lemma_tokens.append(str(lemma_token))
    
    return lemma_tokens


In [ ]:
# Test processing function
print(process("I'm doing well! How about you?"))
# ['im', 'do', 'well', 'how', 'about', 'you']

print(process("Education is the ability to listen to almost anything without losing your temper or your self-confidence."))
# ['education', 'be', 'the', 'ability', 'to', 'listen', 'to', 'almost', 'anything', 'without', 'lose', 'your', 'temper', 'or', 'your', 'self', 'confidence']

print(process("been had done languages cities mice"))
# ['be', 'have', 'do', 'language', 'city', 'mice']

print(process("It's hilarious. Check it out http://t.co/dummyurl"))
# ['it', 'hilarious', 'check', 'it', 'out']

print(process("See it Sunday morning at 8:30a on RTV6 and our RTV6 app. http:…"))
# ['see', 'it', 'sunday', 'morning', 'at', '8', '30a', 'on', 'rtv6', 'and', 'our', 'rtv6', 'app']

print(process("I LOVE usA 🇺🇸!!! Have you gone to the store yet?"))
print(process("It's very chilly outside! Hector's dog will be inside today and for the foreseeable futures...."))

['im', 'do', 'well', 'how', 'about', 'you']
['education', 'be', 'the', 'ability', 'to', 'listen', 'to', 'almost', 'anything', 'without', 'lose', 'your', 'temper', 'or', 'your', 'self', 'confidence']
['be', 'have', 'do', 'language', 'city', 'mice']
['it', 'hilarious', 'check', 'it', 'out']
['see', 'it', 'sunday', 'morning', 'at', '8', '30a', 'on', 'rtv6', 'and', 'our', 'rtv6', 'app']
['i', 'love', 'usa', 'have', 'you', 'go', 'to', 'the', 'store', 'yet']
['it', 'very', 'chilly', 'outside', 'hector', 'dog', 'will', 'be', 'inside', 'today', 'and', 'for', 'the', 'foreseeable', 'future']


In [ ]:
def process_all(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ process all text in the dataframe using process() function.
    Inputs
        df: pd.DataFrame: dataframe containing a column 'Content' loaded from the CSV file
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs
        pd.DataFrame: dataframe in which the values of Content column have been changed from str to list(str),
                        the output from process() function. Other columns are unaffected.
    """

    def apply_process(text):
        if isinstance(text, str):  # Check if the input is a string
            lemma_list = process(text)
            return lemma_list
        else:
            return []

    df['Content'] = df['Content'].apply(apply_process)

    return df

In [ ]:
# Process tweets from training data
train = pd.read_csv("train.csv") 
processed_tweets = process_all(train)

print(processed_tweets.head())

                                             Content    handle
0  [and, i, pledge, to, be, trust, partner, who, ...  Tim_Walz
1  [great, to, be, with, liza, colón, zayas, to, ...  Tim_Walz
2  [kamala, harris, want, to, flood, the, united,...   JDVance
3  [kamala, harris, want, to, raise, tax, on, ame...   JDVance
4  [when, president, trump, be, elect, he, will, ...       GOP


In [ ]:
# derive feature vectors
def identity(x):
    return x
def create_features(processed_tweets, stop_words):
    """ creates the feature matrix using the processed tweet text
    Inputs:
        processed_tweets: pd.DataFrame: processed tweets read from train/test csv file, containing the column 'Content'
        stop_words: list(str): stop_words by nltk stopwords (after processing)
    Outputs:
        sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used
            we need this to tranform test tweets in the same way as train tweets
        scipy.sparse.csr.csr_matrix: sparse bag-of-words TF-IDF feature matrix
    """
    vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False, tokenizer=identity, min_df=2, stop_words=stop_words)
    x = vectorizer.fit_transform(processed_tweets)
    return (vectorizer, x)

In [ ]:
processed_stopwords = list(np.concatenate([process(word) for word in stopwords]))
(tfidf, X) = create_features(processed_tweets["Content"],processed_stopwords)

tfidf, X

/opt/homebrew/Caskroom/miniforge/base/envs/cs-418/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs-418/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  warnings.warn(


(TfidfVectorizer(lowercase=False, min_df=2,
                 stop_words=[np.str_('i'), np.str_('me'), np.str_('my'),
                             np.str_('myself'), np.str_('we'), np.str_('our'),
                             np.str_('ours'), np.str_('ourselves'),
                             np.str_('you'), np.str_('youre'), np.str_('youve'),
                             np.str_('youll'), np.str_('youd'), np.str_('your'),
                             np.str_('yours'), np.str_('yourself'),
                             np.str_('yourselves'), np.str_('he'),
                             np.str_('him'), np.str_('his'), np.str_('himself'),
                             np.str_('she'), np.str_('she'), np.str_('her'),
                             np.str_('hers'), np.str_('herself'), np.str_('it'),
                             np.str_('it'), np.str_('it'), np.str_('itself'), ...],
                 tokenizer=<function identity at 0x169bf14c0>),
 <1125x1365 sparse matrix of type '<class 'numpy.flo

In [ ]:
def create_labels(processed_tweets):
    """ creates the class labels from handle
    Inputs:
        processed_tweets: pd.DataFrame: tweets read from train file, containing the column 'handle'
    Outputs:
        numpy.ndarray(int): dense binary numpy array of class labels
    """
    labels = []
    for index, row in processed_tweets.iterrows():
        if row['handle'] in ['realDonaldTrump', 'JDVance', 'GOP']:
            labels.append(0)
        else:
            labels.append(1)

    return labels

In [ ]:
y = create_labels(processed_tweets)
print(y)

[1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 

In [ ]:
from collections import Counter

class MajorityLabelClassifier():
    """
    A classifier that predicts the mode of training labels
    """
    def __init__(self):
        """
        Initialize your parameter here
        """
        self.mode = -1

    def fit(self, X, y):
        """
        Implement fit by taking training data X and their labels y and finding the mode of y
        i.e. store your learned parameter
        """
        count_0 = y.count(0)
        count_1 = y.count(1)

        if count_0 > count_1:
            self.mode = 0
        else:
            self.mode = 1

    def predict(self, X):
        """
        Implement to give the mode of training labels as a prediction for each data instance in X
        return labels
        """
        return [self.mode] * X.shape[0]

# Get accuracy of classifier by comparing the predicted label of each example to its true label
baselineClf = MajorityLabelClassifier()

# Use fit and predict methods to get predictions and compare it with the true labels y
baselineClf.fit(X,y)
predictions = baselineClf.predict(X)

# Evaluate the classifier (e.g., using accuracy)
from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(y, predictions)
print(train_accuracy)

0.5377777777777778


In [ ]:
def learn_classifier(X_train, y_train, penalty):
    """ learns a classifier from the input features and labels using the penalty function supplied
    Inputs:
        X_train: scipy.sparse.csr.csr_matrix: sparse matrix of features, output of create_features()
        y_train: numpy.ndarray(int): dense binary vector of class labels, output of create_labels()
        penalty: str: penalty function to be used with classifier. [none|l2|l1|elasticnet]
    Outputs:
        sklearn.linear_model.LogisticRegression: classifier learnt from data
    """
    model = None
    if penalty != 'elasticnet':
        model = sklearn.linear_model.LogisticRegression(penalty=penalty, solver='saga')
    else:
        model = sklearn.linear_model.LogisticRegression(penalty=penalty, solver='saga', l1_ratio=0.5)
        
    model.fit(X_train, y_train)

    return model

In [ ]:
classifier = learn_classifier(X, y, 'l2')

In [ ]:
def evaluate_classifier(classifier, X_validation, y_validation):
    """ evaluates a classifier based on a supplied validation data
    Inputs:
        classifier: sklearn.linear_model.LogisticRegression: classifer to evaluate
        X_validation: scipy.sparse.csr.csr_matrix: sparse matrix of features
        y_validation: numpy.ndarray(int): dense binary vector of class labels
    Outputs:
        double: accuracy of classifier on the validation data
    """
    prediction = classifier.predict(X_validation)

    return sklearn.metrics.accuracy_score(y_validation, prediction)

In [ ]:
accuracy = evaluate_classifier(classifier, X, y)
print(accuracy) 

0.9271111111111111


In [19]:
kf = sklearn.model_selection.KFold(n_splits=4, random_state=1, shuffle=True)
kf

KFold(n_splits=4, random_state=1, shuffle=True)

In [ ]:
def best_model_selection(kf, X, y):
    """
    Select the penalty giving best results using k-fold cross-validation.
    Other parameters should be left default.
    Input:
    kf (sklearn.model_selection.KFold): kf object defined above
    X (scipy.sparse.csr.csr_matrix): training data
    y (array(int)): training labels
    Return:
    best_penalty (string)
    """

    # indices = np.array([0, 2])
    # print("X", X[indices, :])
    best_model = ''
    best_accuracy = -1
    print("y type:", type(y))
    for penalty in [None, 'l2', 'l1', 'elasticnet']:
        total_accuracy = 0
         # Use the documentation of KFold cross-validation to split ..
          # training data and test data from create_features() and create_labels()
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index, :], X[test_index, :]

            y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]

            # call learn_classifer() using training split of kth fold
            classifier = learn_classifier(X_train, y_train, penalty)

            # evaluate on the test split of kth fold
            accuracy = evaluate_classifier(classifier, X_test, y_test)
            total_accuracy += accuracy

        # record avg accuracies and determine best model (penalty)
        avg_accuracy = total_accuracy / kf.get_n_splits()
        print(penalty, "accuracy:", avg_accuracy)
        if avg_accuracy > best_accuracy:
            best_accuracy = avg_accuracy
            best_model = penalty    

        
    #return best penalty as string
    return best_model

best_penalty = best_model_selection(kf, X, y)
best_penalty

y type: <class 'list'>
None accuracy: 0.8186788571717019
l2 accuracy: 0.8320177431160243
l1 accuracy: 0.7582847479871786


/opt/homebrew/Caskroom/miniforge/base/envs/cs-418/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs-418/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs-418/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs-418/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


elasticnet accuracy: 0.8071319502284142


'l2'

In [ ]:
def classify_tweets(tfidf, classifier, unlabeled_tweets):
    """ predicts class labels for raw tweet text
    Inputs:
        tfidf: sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used on training data
        classifier: sklearn.linear_model.LogisticRegression: classifier learned
        unlabeled_tweets: pd.DataFrame: tweets read from tweets_test.csv
    Outputs:
        numpy.ndarray(int): dense binary vector of class labels for unlabeled tweets
    """
    processed_unlabeled = process_all(unlabeled_tweets)

    X = tfidf.transform(processed_unlabeled['Content'])

    return classifier.predict(X)

In [ ]:
# Get predictions for unlabelled test data
classifier = learn_classifier(X, y, best_penalty)
unlabeled_tweets = pd.read_csv("test.csv", na_filter=False)
y_pred = classify_tweets(tfidf, classifier, unlabeled_tweets)

*The Logistic Regression classifier did better than the baseline using each penalty. The baseline had an accuracy of 53%, but my classifier when using L2 was 83% which is significantly better.*

In [85]:
unlabeled_tweets['Prediction'] = y_pred

unlabeled_tweets

,Content,Prediction
0,"[i, think, he, know, who, you, be]",1
1,"[will, the, network, that, obsessively, cover,...",0
2,"[i, will, not, raise, tax, on, anyone, make, l...",0
3,"[democrat, republican, and, independent, be, s...",1
4,"[over, the, next, 9, day, let, ’, s, be, inten...",1
...,...,...
194,"[the, american, people, be, exhaust, with, the...",1
195,"[thank, you, for, your, support, mr, president...",1
196,"[great, to, be, in, wilmington, with, north, c...",1
197,"[kamala, harris, just, say, go, to, her, websi...",0
